# Dimensão Tipo Manutenção

Dimensão de tipos de manutenção.

In [ ]:
# Configuração (lê parâmetros do job)
CATALOG = dbutils.widgets.get('catalog') if 'catalog' in [w.key for w in dbutils.widgets.getAll()] else 'manufatura_lakehouse'
SCHEMA_SILVER = dbutils.widgets.get('schema_silver') if 'schema_silver' in [w.key for w in dbutils.widgets.getAll()] else 'silver'
SCHEMA_GOLD = dbutils.widgets.get('schema_gold') if 'schema_gold' in [w.key for w in dbutils.widgets.getAll()] else 'gold'

def fqtn(schema, table):
    if CATALOG and CATALOG.lower() != 'none':
        return f"`{CATALOG}`.`{schema}`.`{table}`"
    else:
        return f"`{schema}`.`{table}`"

# Criar schema se necessário
if CATALOG and CATALOG.lower() != 'none':
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA_GOLD}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA_GOLD}`")


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {fqtn(SCHEMA_GOLD, 'dim_tipo_manutencao')} AS
SELECT DISTINCT
    maintenance_type,
    maintenance_type as description
FROM {fqtn(SCHEMA_SILVER, 'maintenance_orders_clean')}
WHERE maintenance_type IS NOT NULL
""")